In [2]:
import mysql.connector
import csv
import pandas as pd

In [2]:
ls data

SeasonPlayerStats.csv   game.csv                team.csv
common_player_info.csv  officials.csv           team_details.csv


In [18]:
myConnection = mysql.connector.connect( user = 'root',
password = "",
host = 'localhost',
database = 'Basketball')
cursorObject = myConnection.cursor()

In [13]:
season_values = [] 

for i in range(1946, 2024):
    season_id = str(i) 
    season_values.append((season_id,))

season_insert_query = "INSERT INTO Season (Season_ID) VALUES (%s)"
cursorObject.executemany(season_insert_query, season_values)
myConnection.commit()
    

In [19]:
# Read data from team.csv
with open('data/team.csv', mode='r') as team_file:
    team_reader = csv.DictReader(team_file)
    team_values = []
    for team_row in team_reader:
        team_id = team_row['id']
        full_name = team_row['full_name']
        abbreviation = team_row['abbreviation']
        city = team_row['city']
        year_founded = int(float(team_row['year_founded']))
        team_values.append((team_id, full_name, abbreviation, city, year_founded))
     
# Insert the data into the Team table
team_insert_query = "INSERT INTO Team VALUES (%s, %s, %s, %s, %s)"  
cursorObject.executemany(team_insert_query, team_values)
myConnection.commit()

IntegrityError: 1062 (23000): Duplicate entry '1610612737' for key 'team.PRIMARY'

In [20]:
# Read data from officials.csv
with open('data/officials.csv', mode='r') as officials_file:
    officials_reader = csv.DictReader(officials_file)
    officials_values = []
    for row in officials_reader:
        official_id = int(row['official_id'])
        referee_name = f"{row['first_name']} {row['last_name']}"
        officials_values.append((official_id, referee_name))

# Insert the data into the Referees table
insert_officials_query = "INSERT INTO Referees VALUES (%s, %s)"
cursorObject.executemany(insert_officials_query, officials_values)
myConnection.commit()

IntegrityError: 1062 (23000): Duplicate entry '1165' for key 'referees.PRIMARY'

In [16]:
# Read data from game.csv
with open('data/game.csv', mode='r') as game_file:
    game_reader = csv.DictReader(game_file)
    game_values = []
    for row in game_reader:
        game_id = int(row['game_id'])
        season_id = int(row['season_id'])
        game_date = row['game_date']
        season_type = row['season_type']
        home_points = int(float(row['pts_home']))
        away_points = int(float(row['pts_away']))
        home_team_id = int(row['team_id_home'])
        away_team_id = int(row['team_id_away'])
        home_win = home_points > away_points
        game_values.append((game_id, season_id, game_date, season_type, home_points, away_points, home_team_id, away_team_id, home_win))

# Insert the data into the Games table
insert_games_query = "INSERT INTO Games VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
cursorObject.executemany(insert_games_query, game_values)
myConnection.commit()

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`basketball`.`games`, CONSTRAINT `games_ibfk_1` FOREIGN KEY (`Season_ID`) REFERENCES `season` (`Season_ID`))

In [26]:
# Read data from team_details.csv
with open('data/team_details.csv', mode='r') as team_details_file:
    team_details_reader = csv.DictReader(team_details_file)
    coaches_values = []
    coaches_ids = {}
    for row in team_details_reader:
        coach_name = row['headcoach']
        if coach_name:
            coach_id = hash(coach_name) % (10 ** 4)
            coaches_ids[coach_name] = coach_id
            coaches_values.append((coach_id, coach_name))

# Insert the data into the Coaches table
insert_coaches_query = "INSERT INTO Coaches VALUES (%s, %s)"
cursorObject.executemany(insert_coaches_query, coaches_values)
myConnection.commit()


In [37]:
seen = set()

# Read data from team_details.csv
with open('data/team_details.csv', mode='r') as team_details_file:
    team_details_reader = csv.DictReader(team_details_file)
    arenas_values = []
    arena_ids = {}
    abrev_to_arena_id = {}
    for row in team_details_reader:
        arena_id = hash(row['arena']) % (10 ** 4)
        arena_name = row['arena']
        arena_ids[arena_name] = arena_id
        abrev_to_arena_id[row['abbreviation']] = arena_id
        arena_capacity = int(float(row['arenacapacity'])) if row['arenacapacity'] else 0
        if arena_id not in seen:
            arenas_values.append((arena_id, arena_name, arena_capacity))
            seen.add(arena_id)

# Insert the data into the Arenas table
insert_arenas_query = "INSERT INTO Arenas VALUES (%s, %s, %s)"
cursorObject.executemany(insert_arenas_query, arenas_values)
myConnection.commit()

In [9]:
# Read data from common_player_info.csv
with open('data/common_player_info.csv', mode='r') as player_info_file:
    player_info_reader = csv.DictReader(player_info_file)
    player_values = []
    for row in player_info_reader:
        player_id = int(row['person_id'])
        first_name = row['first_name']
        last_name = row['last_name']
        full_name = f"{first_name} {last_name}"
        birth_date = row['birthdate']
        school = row['school']
        country = row['country']
        draft_year = row['draft_year']
        draft_round = row['draft_round']
        if draft_round == '':
            draft_round = None
        player_values.append((player_id, full_name, birth_date, school, country, draft_year, draft_round))

# Insert the data into the Players table
insert_players_query = "INSERT INTO Players VALUES (%s, %s, %s, %s, %s, %s, %s)"
cursorObject.executemany(insert_players_query, player_values)
myConnection.commit()


IntegrityError: 1048 (23000): Column 'Draft_Round' cannot be null

In [32]:
import pandas as pd

# Read data from SeasonPlayerStats.csv
playerseason = pd.read_csv('data/SeasonPlayerStats.csv')

# Function to extract the first year from SEASON_ID
def extract_first_year(season_id):
    return season_id.split('-')[0]

# Apply the function to the SEASON_ID column
playerseason['SEASON_ID'] = playerseason['SEASON_ID'].apply(extract_first_year)

# Filter out the 2023 season and select specific columns
cols = ['PLAYER_ID', 'SEASON_ID', 'TEAM_ID', 'PLAYER_AGE', 'GP', 'PTS', 'REB', 'AST', 'STL', 'BLK']
playerseason = playerseason.loc[playerseason['SEASON_ID'] != "2023"]
playerseason = playerseason.loc[:, cols]

# Handle missing values by filling them with a default value or dropping them
# Example: Filling NA values with 0
playerseason = playerseason.fillna(0)

# Define the data types for each column
dtype_dict = {
    'PLAYER_ID': 'int64',
    'SEASON_ID': 'str',
    'TEAM_ID': 'int64',
    'GP': 'int64',
    'AST': 'int64',
    'REB': 'int64',
    'STL': 'int64',
    'PTS': 'int64',
    'PLAYER_AGE': 'int64',
    'BLK': 'int64',
}

# Convert data types
playerseason = playerseason.astype(dtype_dict)

# Rename columns for clarity
playerseason = playerseason.rename(columns={
    'PLAYER_AGE': 'AGE',
    'GP': 'Games_Played',
    'PTS': 'Points',
    'REB': 'Rebounds',
    'STL': 'Steals',
    'BLK': 'Blocks',
    'AST': 'Assists'
})

# Query the Players table to get all valid PLAYER_IDs
select_players_query = "SELECT Player_ID FROM Players"
cursorObject.execute(select_players_query)
valid_player_ids = set(row[0] for row in cursorObject.fetchall())

# Query the Teams table to get all valid Team_IDS
select_teams_query = "SELECT Team_ID FROM Team"
cursorObject.execute(select_teams_query)
valid_teams_ids = set(row[0] for row in cursorObject.fetchall())

# Filter playerseason to only include rows with valid PLAYER_IDs
playerseason = playerseason[playerseason['PLAYER_ID'].isin(valid_player_ids)]

# Filter playerseason to only include rows with valid Team_IDs
playerseason = playerseason[playerseason['TEAM_ID'].isin(valid_teams_ids)]

# Insert the data into the PlayedSeasonWith table
cols = ['PLAYER_ID', 'SEASON_ID', 'TEAM_ID', 'AGE', 'Games_Played', 'Points', 'Rebounds', 'Assists', 'Steals', 'Blocks']
value_string = '%s, ' * len(cols)
value_string = value_string[:-2]
insert_games_query = f"INSERT INTO PlayedSeasonWith ({', '.join(cols)}) VALUES ({value_string})"
cursorObject.executemany(insert_games_query, playerseason.values.tolist())
myConnection.commit()


In [33]:
# Insert data into CoachesTeam Table
with open('data/team_details.csv', mode='r') as team_details_file:
    team_details_reader = csv.DictReader(team_details_file)
    coaches_team_values = []
    for row in team_details_reader:
        coach_id = coaches_ids[row['headcoach']] if row['headcoach'] else None
        team_id = int(row['team_id'])
        start_date = '2023-01-01'  # Placeholder date
        end_date = '2023-12-31'    # Placeholder date
        coaches_team_values.append((coach_id, team_id, start_date, end_date))

# insert_coaches_team_query = "INSERT INTO CoachesTeam VALUES (%s, %s, %s, %s)"
# cursorObject.executemany(insert_coaches_team_query, coaches_team_values)
# myConnection.commit()

NameError: name 'coaches_ids' is not defined

In [25]:
with open('data/team_details.csv', mode='r') as team_details_file:
    team_details_reader = csv.DictReader(team_details_file)
    home_venue_values = []
    for row in team_details_reader:
        team_id = int(row['team_id'])
        arena_id = arena_ids[(row['arena'])]  # Assuming 'arena' column contains Arena_ID
        home_venue_values.append((team_id, arena_id))

insert_home_venue_query = "INSERT INTO HomeVenue VALUES (%s, %s)"
cursorObject.executemany(insert_home_venue_query, home_venue_values)
myConnection.commit()

In [34]:
with open('data/officials.csv', mode='r') as officials_file:
    officials_reader = csv.DictReader(officials_file)
    officiates_values = []
    for row in officials_reader:
        game_id = int(row['game_id'])
        official_id = int(row['official_id'])
        jersey_number = int(row['jersey_num']) if row['jersey_num'] else None
        officiates_values.append((official_id, game_id, jersey_number))

insert_officiates_query = "INSERT INTO Officiates VALUES (%s, %s, %s)"
cursorObject.executemany(insert_officiates_query, officiates_values)
myConnection.commit()

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`basketball`.`officiates`, CONSTRAINT `officiates_ibfk_1` FOREIGN KEY (`Official_ID`) REFERENCES `referees` (`Official_ID`))

In [ ]:
# TODO: CoachedIn Table
# We don't currently have data on which games a given coach coaches in

In [40]:
# TODO: Look into this further as it is mostly returning None values as we dont know the arena

with open('data/game.csv', mode='r') as game_file:
    game_reader = csv.DictReader(game_file)
    takes_place_in_values = []
    for row in game_reader:
        game_id = int(row['game_id'])
        home_abrev = row['team_abbreviation_home']
        arena_id = abrev_to_arena_id[home_abrev] if home_abrev in abrev_to_arena_id else None
        takes_place_in_values.append((game_id, arena_id))

insert_takes_place_in_query = "INSERT INTO TakesPlaceIn VALUES (%s, %s)"
cursorObject.executemany(insert_takes_place_in_query, takes_place_in_values)
myConnection.commit()


[(24600001, None),
 (24600003, None),
 (24600002, None),
 (24600004, None),
 (24600005, None),
 (24600006, None),
 (24600007, None),
 (24600008, None),
 (24600009, None),
 (24600011, None),
 (24600012, None),
 (24600010, None),
 (24600013, None),
 (24600014, None),
 (24600015, None),
 (24600016, None),
 (24600017, None),
 (24600018, None),
 (24600020, None),
 (24600019, None),
 (24600022, None),
 (24600021, None),
 (24600024, None),
 (24600023, None),
 (24600025, None),
 (24600027, None),
 (24600029, None),
 (24600030, None),
 (24600026, None),
 (24600028, None),
 (24600031, None),
 (24600032, None),
 (24600033, None),
 (24600034, None),
 (24600035, None),
 (24600038, 245),
 (24600037, None),
 (24600036, None),
 (24600041, None),
 (24600039, None),
 (24600040, None),
 (24600042, None),
 (24600046, None),
 (24600045, 245),
 (24600043, None),
 (24600044, None),
 (24600048, None),
 (24600047, None),
 (24600049, None),
 (24600050, None),
 (24600053, None),
 (24600052, None),
 (24600051, No